# Split Videos
Split videos according to timestamp, here into moving and non-moving parts.

In [1]:
import os
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
import sys

sys.path.append('../')

from utils.io import generate_path, load_yaml

servers = load_yaml('config/servers.yml')
server = 'local'

In [2]:
annotations = pd.read_csv('config/endoscopic_datasets_camera_motion_annotation.csv')
annotations

,Name,Group,Video,fps,start / mm:ss:frame,end / mm:ss:frame,camera motion / bool,flags,diff,sum(diff) - length,abr,exp
0,SurgVisDom,SurgVisDom_test_no_indicators,test/test_video_0002.mp4,20,0:00:00,1:16:00,False,type1,1:16:00,0:00:00,lhs,left hand side
1,SurgVisDom,SurgVisDom_test_no_indicators,test/test_video_0002.mp4,20,1:16:00,1:19:00,True,type1,0:03:00,NaN,rhs,right hand side
2,SurgVisDom,SurgVisDom_test_no_indicators,test/test_video_0002.mp4,20,1:19:00,1:47:00,False,type1,0:28:00,NaN,bs,bot side
3,SurgVisDom,SurgVisDom_test_no_indicators,test/test_video_0002.mp4,20,1:47:00,1:57:00,True,type1,0:10:00,NaN,t,top
4,SurgVisDom,SurgVisDom_test_no_indicators,test/test_video_0002.mp4,20,1:57:00,2:50:00,False,type1,0:53:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
634,Mountney,Mountney,left.avi,30,7:19:00,7:24:00,True,default,0:05:00,NaN,NaN,NaN
635,Mountney,Mountney,left.avi,30,7:24:00,8:01:00,False,default,0:37:00,NaN,NaN,NaN
636,Giannarou,Giannarou,left.avi,30,0:00:00,0:47:00,False,default,0:47:00,0:00:00,NaN,NaN
637,Giannarou,Giannarou,left.avi,30,0:47:00,0:51:00,True,default,0:04:00,NaN,NaN,NaN


In [3]:
# convert time string to seconds
def time_string_to_seconds(time_str: str, fps: int):
    r"""Converts time string to seconds.

    Args:
        time_str (str): Time string of format mm:ss:frame
        fps (int): Frames per second

    Returns:
        time (float): Time in seconds
    """
    m, s, f = time_str.split(':')
    time = float(m)*60. + float(s) + float(f)/fps
    print('{} to {}'.format(time_str, time))
    return time

def cut_video(input_video_path: str, output_video_path: str, t1: float, t2: float):
    r"""Cuts video and saves clip.

    Args:
        intput_video_path (str): Path to video
        output_video_path (str): Path to output video
        t1 (float): Beginning of clip in seconds
        t2 (float): End of clip in seconds    
    """
    print(input_video_path)
    with VideoFileClip(input_video_path) as video:
        print(output_video_path)
        new = video.subclip(t1, t2)
        new.write_videofile(output_video_path, codec='libx264')

In [4]:
prefix = servers[server]['database']['location']
output_prefix = servers[server]['database']['location']

name = 'camera_motion_separated'
output_prefix = os.path.join(output_prefix, name)
without_camera_motion_path = 'without_camera_motion'
with_camera_motion_path = 'with_camera_motion'

# generate paths
generate_path(os.path.join(output_prefix, without_camera_motion_path))
generate_path(os.path.join(output_prefix, with_camera_motion_path))

# log origins, flags and names
log = pd.DataFrame(columns=['name', 'group', 'video', 'camera_motion', 'flags', 'origin'])

without_camera_motion_counter = 0
with_camera_motion_counter = 0

# iterate over video annotations
for idx, row in annotations.iterrows():
    # no camera motion
    if row['camera motion / bool'] == False:
        relative_path = '{}/vid_{}.{}'.format(without_camera_motion_path, without_camera_motion_counter, row['Video'][-3:])

        log = log.append({
            'name': name, 
            'group': without_camera_motion_path,
            'video': relative_path, 
            'fps': row['fps'],
            'camera_motion': row['camera motion / bool'],
            'flags': row['flags'],
            'origin': row.to_dict()
        }, ignore_index=True)

        cut_video(
            os.path.join(prefix, row['Name'], row['Video']),
            os.path.join(output_prefix, relative_path),
            time_string_to_seconds(row['start / mm:ss:frame'], row['fps']), time_string_to_seconds(row['end / mm:ss:frame'], row['fps'])
        )

        without_camera_motion_counter += 1
    else:
        relative_path = '{}/vid_{}.{}'.format(with_camera_motion_path, with_camera_motion_counter, row['Video'][-3:])

        log = log.append({
            'name': name, 
            'group': with_camera_motion_path,
            'video': relative_path, 
            'fps': row['fps'],
            'camera_motion': row['camera motion / bool'],
            'flags': row['flags'],
            'origin': row.to_dict()
        }, ignore_index=True)

        cut_video(
            os.path.join(prefix, row['Name'], row['Video']),
            os.path.join(output_prefix, relative_path),
            time_string_to_seconds(row['start / mm:ss:frame'], row['fps']), time_string_to_seconds(row['end / mm:ss:frame'], row['fps'])
        )

        with_camera_motion_counter += 1

log.to_csv(os.path.join(output_prefix, 'log.csv'))
log.to_pickle(os.path.join(output_prefix, 'log.pkl'))